In [1]:
import pandas as pd 
import numpy as np
import time
from sentence_transformers import SentenceTransformer,util
import tensorflow
import pickle

In [10]:
df = pd.read_csv('../fobokiller/data/scrapping_cleaned_sentences.csv',index_col=0)

In [3]:
model = SentenceTransformer('/Users/arthurcollard/code/arthurcol/fobokiller/model')

In [4]:
test=model.encode('I would like to eat a fucking pizza')

In [17]:
embedding = np.load('../embeddings.npz')['arr_0']

In [6]:
util.cos_sim(test,embedding).shape

torch.Size([1, 459349])

In [7]:
embedding[:3000].shape

(3000, 768)

In [8]:
start_time = time.time()

clusters = util.community_detection(embedding[:10000], min_community_size=25, threshold=0.75)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", df.loc[sentence_id,'review_sentences'])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", df.loc[sentence_id,'review_sentences'])


Clustering done after 2.02 sec

Cluster 1, #204 Elements 
	 this place was probably one of the best meals i had while in paris
	 easily one of our favorite restaurants while we were in paris
	 this was one of my top meals in paris
	 ...
	  seriously, if i were to visit paris for one day, this is where i'd be eating
	  i've had their cassoulet à la maison, quite possibly the best i've had in france
	 very cute restaurant located in montmartre area in paris

Cluster 2, #195 Elements 
	  the food and service were excellent
	   service and food was excellent
	 food and service were excellent
	 ...
	  the food was simply fantastic
	  the staff were helpful and friendly
	  very nice staff, accommodating with children and food worth coming back for

Cluster 3, #75 Elements 
	  we had the cheese and potato gratin with duck confit and the duck breast
	  we split the cheese and potato gratin with smoked duck and the duck breast with sauce (i think it was honey)
	  i had the potatoes au gratin wi

In [18]:
embed_tf = tensorflow.convert_to_tensor(embedding)

In [19]:
df['review_sentences'].shape,embed_tf.shape

((459349,), TensorShape([459349, 768]))

In [21]:
text_input = 'I want a good burger'
input_encoded = model.encode(text_input)
similarities = util.cos_sim(input_encoded,np.array(embed_tf))


In [22]:
df_sim = df.assign(sim = similarities.T)

In [23]:
df_sim.sort_values('sim',ascending=False)

,alias,date,rate,review,review_clean,review_sentences,sim
45668,john-viande-paris,2019-08-22,5,"Kourosh F.\nGaithersburg, MD\n5\n75\n184\n8/22...",good good burgers and fries. if your american...,good good burgers and fries,0.756630
381029,breakfast-in-america-2-paris,2017-05-04,4,"Phil B.\nNorth Hollywood, CA\n0\n6\n5/4/2017\n...",the reason yelp exists. what a wonderful disco...,want a proper hamburger,0.756177
162135,h-a-n-d-paris-2,2017-07-04,1,"Madison S.\nNorthbrook, IL\n210\n4\n11\n7/4/20...",absolutly horrid. ordered a burger only to bit...,if you want a burger you're better off going ...,0.733985
234764,pny-burger-oberkampf-paris,2019-06-05,2,"Meg N.\nAshburn, VA\n332\n52\n49\n6/5/2019\nI ...",i love burgers. my go to meal: burger. this pl...,i love burgers,0.729132
355818,the-great-canadian-paris,2013-02-25,3,"Cat W.\nRexford, NY\n0\n5\n2/25/2013\nI was in...",i was in paris with my best friend around east...,"you can get a decent burger, but do you reall...",0.726475
...,...,...,...,...,...,...,...
72279,l-alsace-paris,2018-09-27,1,"Samuel M.\nMarkham, Canada\n27\n2\n5\n9/27/201...",the service is totally unacceptable when their...,we weren't aware of that as no one has explai...,-0.146532
299498,le-relais-de-l-entrecôte-paris-4,2011-02-10,4,"Mike C.\nElite 2021\nSacramento, CA\n908\n1519...",what's not to love...meat being served by fren...,i was pretty covert about it and just used my...,-0.147287
423699,le-relais-de-l-entrecôte-paris-5,2015-06-02,4,"Jeffrey J.\nSan Francisco, CA\n131\n279\n37\n6...",this was a pretty cool experience. it's a no-n...,"they may have been, but they certainly weren'...",-0.149319
187547,breizh-café-paris-2,2017-11-30,4,"Kevin L.\nElite 2021\nLas Vegas, NV\n129\n313\...",after looking for a while on yelp for a good c...,"still, they are experts at their craft",-0.154124


In [39]:
## Create resto df for review count and rate mean 
df_resto = df.groupby('alias').agg({
    'rate':'mean',
    'review':'nunique'
})
df_resto.shape

(1441, 2)

In [40]:
df_resto

,rate,review
alias,,
14-paradis-paris,3.277778,2
142-crêperie-contemporaine-paris-3,4.426531,75
3-pièces-cuisine-paris-2,4.545455,4
42-degrés-paris,4.200000,5
58-tour-eiffel-paris,3.496206,291
...,...,...
zigzag-café-paris,3.968750,12
zino-paris,4.198413,18
zoe-bouillon-paris,5.000000,2


In [34]:
df[df['alias']=='14-paradis-paris']

,alias,date,rate,review,review_clean,review_sentences
49322,14-paradis-paris,2014-02-19,3,"Zeva B.\nParis, France\n1829\n762\n2920\n2/19/...",paradis was far from paradise. at least when i...,paradis was far from paradise
49323,14-paradis-paris,2014-02-19,3,"Zeva B.\nParis, France\n1829\n762\n2920\n2/19/...",paradis was far from paradise. at least when i...,at least when it came to the food which sorel...
49324,14-paradis-paris,2014-02-19,3,"Zeva B.\nParis, France\n1829\n762\n2920\n2/19/...",paradis was far from paradise. at least when i...,"the menu is diverse and appealing, ranging fr..."
49325,14-paradis-paris,2014-02-19,3,"Zeva B.\nParis, France\n1829\n762\n2920\n2/19/...",paradis was far from paradise. at least when i...,"since many of my friends went with the fish, ..."
49326,14-paradis-paris,2014-02-19,3,"Zeva B.\nParis, France\n1829\n762\n2920\n2/19/...",paradis was far from paradise. at least when i...,the egg was way too runny and actually cold o...
49327,14-paradis-paris,2014-02-19,3,"Zeva B.\nParis, France\n1829\n762\n2920\n2/19/...",paradis was far from paradise. at least when i...,i should have sent it back but it had taken s...
49328,14-paradis-paris,2014-02-19,3,"Zeva B.\nParis, France\n1829\n762\n2920\n2/19/...",paradis was far from paradise. at least when i...,"the vegetables were tasty, but not seasoned w..."
49329,14-paradis-paris,2014-02-19,3,"Zeva B.\nParis, France\n1829\n762\n2920\n2/19/...",paradis was far from paradise. at least when i...,"when i asked around the table, everyone felt ..."
49330,14-paradis-paris,2014-02-19,3,"Zeva B.\nParis, France\n1829\n762\n2920\n2/19/...",paradis was far from paradise. at least when i...,"the space, however, is lovely, very low-lit-r..."
49331,14-paradis-paris,2014-02-19,3,"Zeva B.\nParis, France\n1829\n762\n2920\n2/19/...",paradis was far from paradise. at least when i...,"on a friday night the place was packed, and f..."


In [82]:
## create df_filtered to filter with threshold and agg by resto
df_filtered=df_sim[df_sim['sim']>.6]
df_filtered = df_filtered.sort_values('sim',ascending=False).groupby('alias').agg({
    'rate':'mean',
    'review':'count',
    'review_clean':lambda txt : ' // '.join(txt),
    
})

In [86]:
df_output = df_filtered.merge(df_resto,on='alias',how='left',suffixes=('_filtered','_all')).reset_index()

In [87]:
df_output

,alias,rate_filtered,review_filtered,review_clean,rate_all,review_all
0,belushis-paris-3,2.0,1,i was staying at the hostel attached to this b...,2.096000,125
1,big-fernand-paris-12,4.0,1,wow! great burger. in the quest to find the be...,4.025926,270
2,blend-paris-26,4.2,5,"""i still eat a burger at a counter with ketchu...",3.865533,647
3,blend-paris-27,4.0,2,i came across blend while visiting paris a cou...,4.202128,94
4,breakfast-in-america-2-paris,4.0,1,the reason yelp exists. what a wonderful disco...,3.881956,593
...,...,...,...,...,...,...
63,suffren-paris,5.0,1,it's a favorite of my son and his friends. the...,3.565121,453
64,the-great-canadian-paris,3.0,1,i was in paris with my best friend around east...,3.438944,303
65,the-lizard-lounge-paris,4.0,1,burgers are to die for! but that's really all ...,4.230530,321
66,the-moose-paris,3.0,1,nice place to watch nfl. yummy wings and burge...,3.853175,504


In [4]:
from fobokiller.cosim import load_embedding , compute_sim_df ,model

In [5]:
embedding = load_embedding()

2021-11-26 10:58:38.964294: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
df = compute_sim_df('bad pizza',np.array(embedding))
df['ratio']=df['review_filtered']/df['review_all']
#df[df['review_all']>10].sort_values('ratio',ascending=False)
df

TypeError: sequence item 34: expected str instance, float found

In [32]:
df[df['review_all']>10].sort_values('ratio',ascending=False)[:100]

,alias,rate_filtered,review_filtered,review_clean,sim,rate_all,review_all,ratio
88,il-brigante-paris,4.363636,9,excellent pizza. was the first place i ate in ...,0.569076,4.432432,11,0.818182
208,pizza-julia-paris,4.433962,36,really good pizza! and super nice staff. i che...,0.560725,4.538860,60,0.600000
180,maria-luisa-paris-4,4.272727,9,"amazing pizza! best pizza in paris, and wonder...",0.562772,4.046296,15,0.600000
206,pizza-di-loretta-paris,4.875000,7,delicious pizza! especially the mushroom-taste...,0.560004,4.934426,12,0.583333
211,pizza-vesuvio-paris-3,3.705882,9,i used to come here frequently when i lived in...,0.555682,3.520000,17,0.529412
...,...,...,...,...,...,...,...,...
106,la-casa-di-sergio-paris,2.000000,1,this is a cute italian bistro in the heart of ...,0.545420,3.682540,26,0.038462
3,alcazar-paris,3.000000,1,the good: the atmosphere in this place is abso...,0.501369,3.491803,26,0.038462
199,paris-hanoï-paris-6,1.000000,1,very poor meal. we were delivered with deliver...,0.544193,3.965854,27,0.037037
188,nanashi-paris-4,3.000000,1,prettily designed place with friendly staff. c...,0.589520,3.986607,27,0.037037


In [31]:
df.loc[88,'review_clean']

'excellent pizza. was the first place i ate in paris and it proceeded to be the best food i ate all trip. the price is very far for what you are getting (authentic french food). if you choose try this place beware that the sitting is limited. side note: while traveling throughout paris be aware of being pick pocketed. while paris is beautiful it is better when you have all you belongings in the end. // the pizza is decently good. the servers are too nice bc they are swamped. the avg wait time for a table is super long (.hrs) if you didn\'t make a reservation on a saturday night for a table of two at h. to go orders are at least mins. it would be better to order on deliveries or foodera. the service makes me not want to go back. // excellent pizza, made with fresh ingredients! the place is quite simple, but it\'s charming. the owen is almost in the middle of the room, you can see all the pizzaiolo\'s creations! must go place if you like pizza!!! amazing!! // the pizza itself at this pla

In [13]:
def compute_sim_df(text,embedding,n_prox=None,min_review=0):
    input_encoded = model.encode(text)
    similarities = util.cos_sim(input_encoded, np.array(embedding))

    df_sim = df.assign(sim = similarities.T)

    if not n_prox:
        df_filtered = df_sim.sort_values('sim',ascending=False)\
            .groupby('alias').agg({ 'rate':'mean',
                                    'review':'nunique',
                                    'review_clean':lambda txt : ' // '.join(str(txt)),
                                    'sim':'mean'
                                    })

        df_output = df_filtered.merge(df_resto,on='alias',how='left',suffixes=('_filtered','_all')).reset_index()

        return df_output[df_output['review_all']>min_review]

    if n_prox:
        df_filtered= df_sim.sort_values('sim',ascending=False)[:n_prox]\
            .groupby('alias').agg({ 'rate':'mean',
                                    'review':'nunique',
                                    'review_clean':lambda txt : ' // '.join(str(txt)),
                                    'sim':'mean'
                                    })

        df_output = df_filtered.merge(df_resto,on='alias',
                                      how='left',suffixes=('_filtered','_all')).reset_index()

        return df_output[df_output['review_all'] > min_review]


In [14]:
compute_sim_df('pizza du feu',embedding,n_prox=None,min_review=0)

NameError: name 'df_resto' is not defined